In [324]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### https://adventofcode.com/2021/day/1

In [325]:
# read in depths

df1=pd.read_csv('day1_1.txt', header=None)
df1.columns=['val']
df1.tail()

,val
1995,9830
1996,9831
1997,9841
1998,9860
1999,9875


In [326]:
# Part 1 - calc diff from previous value
df1['calc1']=df1.val-df1.val.shift(1)
df1['result1']=df1.calc1.apply(lambda x:'increase' if x>0 else 'x')
df1.result1.value_counts()

increase    1722
x            278
Name: result1, dtype: int64

In [327]:
# Part 2 - create column of rolling sum of 3 then calc diffs
df1['three']=df1.val + df1.val.shift(-1) + df1.val.shift(-2)
df1['calc2']=df1.three-df1.three.shift(1)
df1['result2']=df1.calc2.apply(lambda x:'increase' if x>0 else 'x')
df1.result2.value_counts()

increase    1748
x            252
Name: result2, dtype: int64

### https://adventofcode.com/2021/day/2

In [328]:
df2=pd.read_csv('day2.txt', header=None)
df2.columns=['val']
df2.tail()

,val
995,forward 9
996,down 3
997,down 7
998,down 5
999,forward 7


In [329]:
# split instructions and values - group and sum
df2['instr']=df2.val.apply(lambda x:x.split(' ')[0])
df2['num']=df2.val.apply(lambda x:float(x.split(' ')[1]))
dfx=pd.DataFrame(df2.groupby('instr').num.sum().reset_index(name='tot'))
dfx

,instr,tot
0,down,2046.0
1,forward,1990.0
2,up,1046.0


In [330]:
((dfx[dfx.instr=='down'].tot.values-dfx[dfx.instr=='up'].tot.values) * dfx[dfx.instr=='forward'].tot.values)[0]

1990000.0

In [331]:
# part 2 - brute force  loop through and calc values at each stage

aim=0
h=0
d=0
for idx, row in df2.iterrows():
    if row.instr == 'up':
        aim-=row.num 
    if row.instr == 'down':
        aim+=row.num
    if row.instr == 'forward':
        h+=row.num
        d+=aim * row.num

print(h, d, h*d)

1990.0 992674.0 1975421260.0


### https://adventofcode.com/2021/day/3

In [332]:
# read in and convert to dec

df3=pd.read_csv('day3.txt',header=None)
df3.columns=['bin']
df3['dec']=df3.bin.apply(lambda x:int(str(x),2))
dfT=df3.sort_values('dec',ascending=False)[:5]

In [333]:
# epsilon = 111111.. - gamma
maxno=2**12-1
maxno

4095

In [334]:
# find gamma - loop through 2**0 t0 2**11 and do bitwise comparison of dec values

def bitand(dnum,n):
    if dnum & 2**n !=0:
        return 1
    else:
        return 0

gamma=0
for i in range(12):
    bits=df3.dec.apply(bitand, n=i).sum()
    if bits>500:
        gamma+=2**i

eps=maxno-gamma
gamma*eps

4118544

In [335]:
# Part 2 - order is reversed compared to part 1

# oxygen 1

df=df3.copy()
for i in range(11,-1,-1):
    bit=df.dec.apply(bitand, n=i)
    if bit.sum()/len(df) >= .5:
        df=df[bit==1]
    else:
        df=df[bit==0]
    
    if len(df)==1:
        oxygen=df.dec.values[0]
        break

#  CO2 0
df=df3.copy()
for i in range(11,-1,-1):
    bit=df.dec.apply(bitand, n=i)
    if bit.sum()/len(df) <.5:
        df=df[bit==1]
    else:
        df=df[bit!=1]
    if len(df)==1:
        co2=df.dec.values[0]
        break

print(oxygen, co2, oxygen*co2)   
    

1527 2510 3832770


## https://adventofcode.com/2021/day/4

In [336]:
# load in called numbers

with open('day4.txt') as f:
    numbers=f.readline().replace('\n','').split(',')
print(*numbers)  
    

6 69 28 50 36 84 49 13 48 90 1 33 71 0 94 59 53 58 60 96 30 34 29 91 11 41 77 95 17 80 85 93 7 9 74 89 18 25 26 8 87 38 68 5 12 43 27 46 62 73 16 55 22 4 65 76 54 52 83 10 21 67 15 47 45 40 35 66 79 51 75 39 64 24 37 72 3 44 82 32 78 63 57 2 86 31 19 92 14 97 20 56 88 81 70 61 42 99 23 98


In [337]:
# load rest of the data into an array

data=np.loadtxt("day4.txt", dtype=np.uint8, skiprows=1)
data.shape

(500, 5)

In [338]:
# reshape into 100 5x5 cards

cards=data.copy().reshape(100,5,5)
cards[:1]

array([[[22, 59,  7, 10,  6],
        [33, 36, 96, 55, 23],
        [13, 85, 18, 29, 28],
        [75, 46, 83, 73, 58],
        [34, 40, 87, 56, 98]]], dtype=uint8)

In [339]:
#  make 100 blank cards and mark with 1 in the corresponding position of each matched nyumber
cards=data.reshape(100,5,5)
results=np.zeros((100,5,5))
for draw in numbers:
        draw=int(draw)
        results[(np.where(cards==draw))]=1
        if  np.any(results.sum(axis=1)==5):
            cardnum=np.where(results.sum(axis=1)==5)[0][0]
            print(draw, cardnum)
            break
        if np.any(np.transpose(results).sum(axis=0)==5):
            cardnum=np.where(np.transpose(results).sum(axis=0)==5)[0][0]
            print(draw, cardnum)
            break


91 69


In [340]:
# sum of winning card numbers not matched times last number drawn
(1*np.logical_not(results[cardnum])*cards[cardnum]).sum() * draw

71708

In [341]:
wins=[n for n in range(100)]
cards=data.reshape(100,5,5)
results=np.zeros((100,5,5))
for draw in numbers:
        draw=int(draw)
        results[(np.where(cards==draw))]=1
        for f in range(100):
                if max(results[f].sum(axis=0))==5 or max(results[f].sum(axis=1))==5:
                    remember=results[f].copy()
                    results[f]=0
                    if len(wins)==1:
                        print(f)
                        print(draw)
                        print((1*np.logical_not(remember)*cards[f]).sum()*draw)
                    if f in wins:
                        wins.remove(f)
                    
                    
                    


67
86
56330
9
97
34726


## https://adventofcode.com/2021/day/5

In [342]:
# laod and tidy data
df=pd.read_csv('day5.txt', header=None)
df.columns=['a','x','d']
df['b']=df.x.apply(lambda x:int(x.split('-')[0]))
df['c']=df.x.apply(lambda x:int(x.split('>')[1]))
df=df[['a','b','c','d']].copy()
df.sample(5)

,a,b,c,d
136,450,86,489,86
304,809,156,129,836
200,43,949,945,47
406,328,289,280,289
416,188,793,761,793


In [343]:
# check dims
print(df.max())
print( df.min())

a    989
b    989
c    989
d    988
dtype: int64
a    10
b    12
c    10
d    10
dtype: int64


In [344]:
# horiz and vert only
df1=df[(df.a == df.c)|(df.b==df.d)].copy()
len(df1)

353

In [345]:
df1['x']=np.sign(df1.c-df1.a)
df1['y']=np.sign(df1.d-df1.b)
df1.sample(3)

,a,b,c,d,x,y
251,652,638,652,569,0,-1
91,375,406,200,406,-1,0
419,510,254,510,862,0,1


In [346]:

res=results=np.zeros((1000,1000))
for idx, row in df1.iterrows():
    if row.x==1:
        for f in range(row.a,1+row.c):
            res[f,row.b]+=1
    if row.x==-1:
        for f in range(row.c,1+row.a):
            res[f,row.b]+=1
    if row.y==1:
        for f in range(row.b,1+row.d):
            res[row.a,f]+=1
    if row.y==-1:
        for f in range(row.d,1+row.b):
            res[row.a,f]+=1
   
(1*(res>1)).sum() 

7085

In [347]:

dfx=df.merge(df1,how='left')
dfd=dfx[dfx.x.isnull()].copy()
dfd['x']=np.sign(dfd.c-dfd.a)
dfd['y']=np.sign(dfd.d-dfd.b)
dfd.sample(3)

,a,b,c,d,x,y
76,875,930,26,81,-1,-1
77,875,115,127,863,-1,1
2,340,734,774,300,1,-1


In [348]:

for idx, row in dfd.iterrows():
    xc=row.a
    yc=row.b
    for i in range(1+max(row.a,row.c)-min(row.a,row.c)):
        res[xc,yc]+=1
        xc+=1*row.x
        yc+=1*row.y

(1*(res>1)).sum() 

20271

## https://adventofcode.com/2021/day/6

In [349]:
with open('day6.txt','r') as f:
    s = f.readlines()


In [350]:
# days = 80
days = 256

dctLantern={}
st=len(str(s).split(','))
for i in range(0,9):
    dctLantern[str(i)]=len(str(s))-len(str(s).replace(str(i),''))

    
for x in range(256):
    dctNextDay={}
    
    
    for i in range(0,8):
        dctNextDay[str(i)]=dctLantern[str(i+1)]
        val=dctLantern['0']
    dctNextDay['8']=val
    dctNextDay['6']+=val
    dctLantern=dctNextDay

dctLantern

{'0': 141148347315,
 '1': 162057469235,
 '2': 171485519495,
 '3': 188293726992,
 '4': 209248181091,
 '5': 219291507258,
 '6': 252739136348,
 '7': 118233178535,
 '8': 137808935019}

In [351]:
tot=0
for i in range(9):
    tot+=dctLantern[str(i)]

tot

1600306001288